This visualisation requires multiples libraries to be installed including plotly and dash.
Once launched, it will create a server on the local host where the visualisation will be shown.

### This visualisation is composed of two main representations :
 - a choropleth map showing the distribution of names across departments ;
 - a stacked bar chart showing the 10 most frequent names.

One of these visualisations acts as 'input' and filters the second one, the 'output' By default, the 'input' visualisation is the choropleth but the user can change it if he or she so wishes it.

 - If the choropleth map is the 'input', then by clicking on a department, the stacked bar chart will show the ten most frequent names in the chosen department ;
 - If the stacked bar chart is the 'input', then by clicking on a name, the choropleth map will show the distribution of the chosen name across all departments.
 
As 'input', there exist two versions for the choropleth map :
 - 'Population' shows the number of births by department ;
 - 'Max names' shows the most frequent name by department. 

There also exist multiple filters :
 - A filter by gender (girls, boys or both) ;
 - A filter by name : the user can choose only to keep names beginning with 'A' ;
 - A filter by years : the user choose a start and end year and only names between this interval shall be kept.

This visualisation allows the user to quickly see which names are the most popular in the country, their distribution as well as the presence in some departments (chosen by the user) of more 'unique' names.

For instance, the user can choose as 'input' the bar chart and filter the data by keeping only girls. If he clicks on 'Marie', it is shown that 'Marie' is mostly present in the western as well as northernmost and easternmost parts of France. Despite differences of population, there are 25K more 'Marie' in 'Finistère' than in 'Paris'. The same can be said of the name 'Anne'. Conversely for another popular name such as 'Catherine', the distribution is much more uniform.

If the user chooses as 'input' Map (Max names) and filters the data by keeping only boys, then we can see that there are only 4 unique names that are the most frequent names in a department with 'Jean' being crealy the most important. However, if we filter with years and keep only the decade 2010-2020, then we can see a far greater degree of diversity with 13 different names with some regional particularities : 'Jules' being mostly present in the eastern part of France.


#### Addendum: Due to some technical difficulties, the option to filter the barchat while using 'Max names' choropleth map is disabled. Moreover, filtering by name can take a bit of time.

In [ ]:
import geojson
import numpy as np
import pandas as pd
import plotly.express as px

In [ ]:
df = pd.read_csv('dpt2020.csv', sep=';')

df = df[df['dpt'] != 'XX']
df = df[df['dpt'] != '971']
df = df[df['dpt'] != '972']
df = df[df['dpt'] != '973']
df = df[df['dpt'] != '974']

df['sexe'] = df['sexe'].apply(str)

df.loc[df['sexe'] == '1', 'sexe'] = 'M'
df.loc[df['sexe'] == '2', 'sexe'] = 'F'

df = df[df['preusuel'] != '_PRENOMS_RARES']

df['annais'] = df['annais'].astype(int)

df.head()

In [ ]:
min_year, max_year = df['annais'].min(), df['annais'].max()
marks_year = list(range(df['annais'].min(), df['annais'].max() + 1, 10))
marks_year = {i:str(i) for i in marks_year}

In [ ]:
with open('Departements.geojson') as f:
    geojson_departments = geojson.load(f)

for idx, department in enumerate(geojson_departments['features']):
    department['id'] = department['properties']['code']

In [ ]:
def df_filter_gendered_names(df, gendered_names):
    tuples_in_df = pd.MultiIndex.from_frame(df[['preusuel', 'sexe']])
    df = df[tuples_in_df.isin(gendered_names)]
    return df

In [ ]:
def df_filter_partial_name(df, partial_name):
    df = df[df['preusuel'].str.startswith(partial_name)]
    return df

In [ ]:
def df_filter_names(df, names):
    df = df[df['preusuel'].isin(names)]
    return df

In [ ]:
def df_filter_genders(df, genders):
    df = df[df['sexe'].isin(genders)]
    return df

In [ ]:
def df_filter_departments(df, departments):
    df = df[df['dpt'].isin(departments)]
    return df

In [ ]:
def df_filter_years(df, year_start=None, year_end=None):
    if year_start is not None:
        df = df[df['annais'] >= year_start]
        
    if year_end is not None:
        df = df[df['annais'] <= year_end]
        
    return df

In [ ]:
def df_groupy(df, columns):
    df = df.groupby(columns, as_index=False)['nombre'].sum()
    return df

In [ ]:
def get_top(df, n=10):
    df = df.sort_values('nombre', ascending=False).reset_index(drop=True).head(10)
    return df

In [ ]:
def get_genders(genders):
    if genders == 'Boys':
        return ['M']
    elif genders == 'Girls':
        return ['F']
    else:
        return ['M', 'F']

In [ ]:
visible_true = {'width': '49%', 'display': 'inline-block', 'padding': '0 20'}
visible_false = {'width': '49%', 'display': 'none', 'padding': '0 20'}

In [ ]:
from dash import Dash, html, dcc, Input, Output

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div([
    
    html.Div([
        dcc.Dropdown(
            ['Boys and Girls', 'Boys', 'Girls'],
            'Boys and Girls',
            id='gender-selection'
        )
    ], style={'width': '14%', 'display': 'inline-block'}),
    
    html.Div([
        dcc.Input(
            type='text',
            value=None,
            placeholder='Filter names',
            id='input_name'  
        )
    ], style={'width': '24%', 'display': 'inline-block', 'vertical-align':'top'}),
    
    html.Div([
        dcc.Dropdown(
            ['Input: Map (Population)', 'Input: Map (Max name)', 'Input: Barchart'], 
            'Input: Map (Population)', 
            id='input-choice'
        ),
        
    ], style={'width': '14%', 'display': 'inline-block'}),
    
    html.Div([
        dcc.RangeSlider(
            min=min_year, max=max_year, step=10,
            value = [min_year, max_year],
            marks=marks_year,
            id='year-slider'
        )
    ], style={'width': '44%', 'display': 'inline-block'}), 
    
    html.Div([
        dcc.Graph(
            id='bar'
        )
    ], style={'width': '49%', 'display': 'inline-block', 'padding': '0 20'}),
    
    html.Div([
        dcc.Graph(
            id='map-pop'
        )
    ], id='div-map-pop', style={'width': '49%', 'display': 'inline-block', 'padding': '0 20'}),
    
    html.Div([
        dcc.Graph(
            id='map-max'
        )
    ], id='div-map-max', style={'width': '49%', 'display': 'inline-block', 'padding': '0 20'}),
])


@app.callback(
    Output('map-pop', 'figure'),
    Input('gender-selection', 'value'), 
    Input('input_name', 'value'),
    Input('year-slider', 'value'),
    Input('input-choice', 'value'),
    Input('bar', 'selectedData')
)
def update_map_pop(gender_choice, input_name, pair_years, choice_input, selectedData):
    df_copy = df.copy()
    
    genders = get_genders(gender_choice)
    
    df_copy = df_filter_genders(df_copy, genders=genders)
    df_copy = df_filter_years(df_copy, year_start=pair_years[0], year_end=pair_years[1])
    
    if input_name is not None:
        df_copy = df_filter_partial_name(df_copy, partial_name=input_name.upper())
    
    if choice_input == 'Input: Barchart':
        df_copy = df_groupy(df_copy, columns=['preusuel', 'sexe', 'dpt'])

        if selectedData is not None:
            gendered_names = []
            for point in selectedData['points']:
                gendered_name = (point['x'], point['customdata'][0])
                gendered_names.append(gendered_name)
            df_copy = df_filter_gendered_names(df_copy, gendered_names=gendered_names)
    
    df_copy = df_groupy(df_copy, columns=['dpt'])

    fig = px.choropleth_mapbox(df_copy, geojson=geojson_departments, 
                               locations='dpt', 
                               color='nombre', 
                               color_continuous_scale='Turbo',
                               mapbox_style='carto-positron',
                               zoom=4, 
                               center = {"lat": 46, "lon": 2},
                               opacity=0.5)
    
    fig.update_layout(
        margin={"r":0,"t":0,"l":0,"b":0}, 
        clickmode='event+select'
    )
    
    return fig


@app.callback(
    Output('map-max', 'figure'),
    Input('gender-selection', 'value'), 
    Input('input_name', 'value'),
    Input('year-slider', 'value'),
    Input('input-choice', 'value'))
def update_map_max(gender_choice, input_name, pair_years, choice_input):
    df_copy = df.copy()
    
    genders = get_genders(gender_choice)
    
    df_copy = df_filter_genders(df_copy, genders=genders)
    df_copy = df_filter_years(df_copy, year_start=pair_years[0], year_end=pair_years[1])
    
    if input_name is not None:
        df_copy = df_filter_partial_name(df_copy, partial_name=input_name.upper())
        
    df_copy = df_groupy(df_copy, columns=['preusuel', 'dpt'])
    df_copy = df_copy.loc[df_copy.groupby('dpt')['nombre'].idxmax()]

    fig = px.choropleth_mapbox(df_copy, geojson=geojson_departments, 
                               locations='dpt', 
                               color='preusuel', 
                               mapbox_style='carto-positron',
                               color_discrete_sequence = px.colors.qualitative.Dark24,
                               zoom=4,
                               center = {"lat": 46, "lon": 2},
                               opacity=0.5)
    
    fig.update_layout(
        margin={"r":0,"t":0,"l":0,"b":0}
    )
    
    return fig


@app.callback(
    Output('div-map-pop', 'style'),
    Output('div-map-max', 'style'),
    Input('input-choice', 'value'))
def update_div_map(choice_input):
    df_copy = df.copy()
    
    div_map_pop = visible_true if (choice_input == 'Input: Map (Population)') or (choice_input == 'Input: Barchart') else visible_false
    div_map_max = visible_true if choice_input == 'Input: Map (Max name)' else visible_false
    
    return div_map_pop, div_map_max


@app.callback(
    Output('bar', 'figure'), 
    Input('gender-selection', 'value'),
    Input('input_name', 'value'),
    Input('year-slider', 'value'),
    Input('input-choice', 'value'),
    Input('map-pop', 'selectedData'))
def update_bar(gender_choice, input_name, pair_years, choice_input, data_map_pop):
    df_copy = df.copy()
    genders = get_genders(gender_choice)
    
    df_copy = df_filter_genders(df_copy, genders=genders)
    df_copy = df_filter_years(df_copy, year_start=pair_years[0], year_end=pair_years[1])
    
    if input_name is not None:
        df_copy = df_filter_partial_name(df_copy, partial_name=input_name.upper())
    
    
    if choice_input == 'Input: Map (Population)':
        if data_map_pop is not None:
            locations = []
            for points in data_map_pop['points']:
                location = points['location']
                locations.append(location)   
            df_copy = df_filter_departments(df_copy, departments=locations)
    
    df_copy = df_groupy(df_copy, columns=['preusuel', 'sexe'])
    df_copy = get_top(df_copy, n=10)
    
    fig = px.bar(df_copy, 
                 x='preusuel', 
                 y='nombre', 
                 color='sexe', 
                 custom_data=['sexe'],
                 color_discrete_map={'F':'gold', 'M':'darkblue'})
    
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0}, xaxis={'categoryorder':'total descending'})
    
    if choice_input == 'Input: Barchart':
        fig.update_layout(clickmode='event+select')
    
    return fig

app.run_server()